In [ ]:
import torchvision 
import torch 
from PIL import Image 

import os 
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw 
font = ImageFont.truetype("/usr/share/fonts/truetype/noto/NotoSansMono-Regular.ttf", 32)
import numpy as np
from tqdm.auto import tqdm
import skimage
from tqdm.auto import tqdm

In [5]:
OBJECTS = [
    'shoe',
]
SCALES = [-1.0, 1.0]
LORA_CHKPT = range(100, 40000, 100)
#SEEDS = [807, 200, 201, 202, 800]
SEEDS = range(0,10000, 100)
LEARNING_RATE = "5e-5"
#ROOT_DIR = "../output/chkpt100/512_unsplash250_cast_learning_rate/"
ROOT_DIR = "../output/chkpt100/512_unsplash250_cast_learning_rate/ckpt10000_5e-5_bottle_minus1"
# COLOR_TYPE="gray"
# FOLDER_OUT = f"{ROOT_DIR}/{LEARNING_RATE}/{COLOR_TYPE}_with_text"
# FOLDER_IN = f"{ROOT_DIR}/{LEARNING_RATE}/{COLOR_TYPE}"
# FOLDER_VID = f"{ROOT_DIR}/{LEARNING_RATE}/{COLOR_TYPE}_video"
# FOLDER_ROW = f"{ROOT_DIR}/{LEARNING_RATE}/{COLOR_TYPE}_row"

COLOR_TYPE="gray"
FOLDER_OUT = f"{ROOT_DIR}/{COLOR_TYPE}_with_text"
FOLDER_IN = f"{ROOT_DIR}/{COLOR_TYPE}"
FOLDER_VID = f"{ROOT_DIR}/{COLOR_TYPE}_video"
FOLDER_ROW = f"{ROOT_DIR}/{COLOR_TYPE}_row"
os.makedirs(FOLDER_OUT, exist_ok=True)
os.makedirs(FOLDER_VID, exist_ok=True)
os.makedirs(FOLDER_ROW, exist_ok=True)

In [6]:


for lora_id, lora_name in enumerate(tqdm(LORA_CHKPT)):
    for prompt_id, prompt_name in enumerate(OBJECTS):
        for scale_id, scale_name in enumerate(SCALES):
            for seed_id, seed_name in enumerate(SEEDS):
                fpath = f"{FOLDER_IN}/{lora_id:04d}_{prompt_id:04d}_{scale_id:04d}_{seed_id:04d}.png"
                #print(fpath)
                image = Image.open(fpath)
                draw = ImageDraw.Draw(image)
                draw.text((10, 10),f"step: {lora_name:5d}\nprompt: {prompt_name}\nscale: {scale_name:.2f}\nseed: {seed_name:3d}\nLR: {LEARNING_RATE}",(255,255,255),font=font)
                image.save(f"{FOLDER_OUT}/{lora_id:04d}_{prompt_id:04d}_{scale_id:04d}_{seed_id:04d}.png")



  0%|          | 0/399 [00:00<?, ?it/s]

../output/chkpt100/512_unsplash250_cast_learning_rate//5e-5/gray/0000_0000_0000_0000.png
../output/chkpt100/512_unsplash250_cast_learning_rate//5e-5/gray/0000_0000_0000_0001.png
../output/chkpt100/512_unsplash250_cast_learning_rate//5e-5/gray/0000_0000_0000_0002.png
../output/chkpt100/512_unsplash250_cast_learning_rate//5e-5/gray/0000_0000_0000_0003.png
../output/chkpt100/512_unsplash250_cast_learning_rate//5e-5/gray/0000_0000_0000_0004.png
../output/chkpt100/512_unsplash250_cast_learning_rate//5e-5/gray/0000_0000_0001_0000.png
../output/chkpt100/512_unsplash250_cast_learning_rate//5e-5/gray/0000_0000_0001_0001.png
../output/chkpt100/512_unsplash250_cast_learning_rate//5e-5/gray/0000_0000_0001_0002.png
../output/chkpt100/512_unsplash250_cast_learning_rate//5e-5/gray/0000_0000_0001_0003.png
../output/chkpt100/512_unsplash250_cast_learning_rate//5e-5/gray/0000_0000_0001_0004.png
../output/chkpt100/512_unsplash250_cast_learning_rate//5e-5/gray/0001_0000_0000_0000.png
../output/chkpt100/51

KeyboardInterrupt: 

In [ ]:
for lora_id, lora_name in enumerate(tqdm(LORA_CHKPT)):
    for scale_id, scale_name in enumerate(SCALES):
        images = []
        for prompt_id, prompt_name in enumerate(OBJECTS):
            for seed_id, seed_name in enumerate(SEEDS):
                fpath = os.path.join(FOLDER_OUT, f'{lora_id:04d}_{prompt_id:04d}_{scale_id:04d}_{seed_id:04d}.png')
                image = torchvision.io.read_image(fpath)
                image = torchvision.transforms.functional.resize(image, (256,256))
                images.append(image)
        images = torch.stack(images)
        #grid = torchvision.utils.make_grid(images, nrow=len(SEEDS))
        grid = torchvision.utils.make_grid(images, nrow=100)
        grid = torchvision.transforms.ToPILImage()(grid)
        output_name = f"{FOLDER_ROW}/{lora_id:04d}_{scale_id:04d}.png"
        grid.save(output_name)


  0%|          | 0/20 [00:00<?, ?it/s]

/home/vll/venv_pytorch2.0/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [ ]:
#FOLDER_VID = f"../output/chkpt100/unsplash250_sh1/gray_row2000"
#FOLDER_ROW = f"../output/chkpt100/unsplash250_sh1/gray_video2000"
#os.makedirs(FOLDER_VID, exist_ok=True)
#os.makedirs(FOLDER_ROW, exist_ok=True)
SCALE_ID = [0, 1]
for scale_id in (SCALE_ID):
    cmd = f'ffmpeg -r 5 -i "{FOLDER_ROW}/%04d_{scale_id:04d}.png" -c:v libx264 -crf 12 -pix_fmt yuv420p -vf pad="width=ceil(iw/2)*2:height=ceil(ih/2)*2" {FOLDER_VID}/{scale_id:04d}.mp4'
    os.system(cmd)



ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab